In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import logging
import pandas as pd
import numpy as np
import yaml

sys.path.append(os.path.dirname(os.path.abspath('')))

from stratification.params import SplitBuilderParams
from prepilot.params import PrepilotParams
from prepilot.prepilot_experiment_builder import PrepilotExperimentBuilder
from prepilot.prepilot_split_builder import PrepilotSplitBuilder
from auto_ab.abtest import ABTest
from auto_ab.params import ABTestParams

logging.basicConfig(level = logging.INFO)

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('ab_data.csv')

with open("../auto_ab/configs/auto_ab.config.yaml", "r") as stream:
    try:
        ab_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [3]:
df["moda_city"] = np.random.randint(1, 5, df.shape[0])
df["moda_city"] = df["moda_city"].astype(str)
df["country"] = np.random.randint(1, 3, df.shape[0])
df["id"] = df.index

df["numerator"] = np.random.randint(1, 5, df.shape[0])
df["denominator"] = np.random.randint(1, 5, df.shape[0])
df["country"] = np.random.randint(1, 3, df.shape[0])
df['conversion'] = np.where(df.noise_now<0,0,1)

In [5]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [8]:
df.to_csv('ab_data.csv', index=False)

In [ ]:
df.to_csv()

In [3]:
from auto_ab.params import *

data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    strata_col='country', 
    target='height_now', 
    target_flg='bought', 
    predictors=['weight_now'], 
    numerator='clicks', 
    denominator='sessions', 
    covariate='height_prev', 
    target_prev='height_prev', 
    predictors_prev=['weight_prev'], 
    cluster_col='kl-divergence', 
    clustering_cols=['col1', 'col2', 'col3'], 
    is_grouped=True
)

simulation_params = SimulationParams(
    n_iter=100, 
    split_rates=[0.1, 0.2, 0.3, 0.4, 0.5], 
    vars=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0], 
    extra_params=[]
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided', 
    split_ratios=(0.5, 0.5), 
    strategy='simple_test', 
    strata='country', 
    strata_weights={'US': 0.8, 'UK': 0.2}, 
    metric_type='solid', 
    metric_name='mean', 
    metric=np.mean, 
    n_boot_samples=200, 
    n_buckets=50
)

result_params = ResultParams(
    to_csv=True, 
    csv_path='/app/data/internal/guide/solid_mde.csv'
)

splitter_params = SplitterParams(
    split_rate=0.5, 
    name='default'
)

ab_params = ABTestParams(data_params,simulation_params,hypothesis_params,result_params,splitter_params)

In [4]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': None,
        'target': None
    },
    region_col = "moda_city",
    split_metric_col = "height_now",
    customer_col = "id",
    cols = [],
    cat_cols=[
        
    ],
    pvalue=0.05,
    n_top_cat=100,
    stat_test="ttest_ind"
)

# Препилот для числовой переменной

In [5]:
prepilot_params = PrepilotParams(
    metrics_names=['height_now'],
    injects=[1.0001,1.0002,1.0003,1.0004,1.00042,1.00044,1.00046,1.00048,1.0005,1.001],
    min_group_size=35000, 
    max_group_size=50000, 
    step=5000,
    variance_reduction = None,
    use_buckets = False,
    stat_test = ABTest.test_hypothesis_boot_confint,
    iterations_number = 50,
    max_beta_score=0.9,
    min_beta_score=0.02,
)

In [6]:
prepilot = PrepilotExperimentBuilder(df, ab_params,
                                     prepilot_params,
                                     split_builder_params)

In [7]:
beta,alpha = prepilot.collect()

INFO:stratification.split_builder:Calculate stratas for guest table
INFO:stratification.binning:27017 outliers found
INFO:stratification.split_builder:control: Desired size = 0.175,             resulting size = 34998, diff = 0.01 %
INFO:stratification.split_builder:target: Desired size = 0.21211864098616987,             resulting size = 35001, diff = 0.0 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_builder:control: Desired size = 0.175,             resulting size = 34998, diff = 0.01 %
INFO:stratification.split_builder:target: Desired size = 0.21211864098616987,             resulting size = 35001, diff = 0.0 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_builder:control: Desired size = 0.175,             resulting size = 34998, diff = 0.01 %
INFO:stratification.split_builder:target: Desired size = 0.21211864098616987,             resulting size = 35001, diff = 0.0 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_bu

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [9]:
beta

split_rate        (35000, 35000) (40000, 40000) (45000, 45000) (50000, 50000)
metric     MDE                                                               
height_now 0.01%           >=0.9          >=0.9          >=0.9          >=0.9
           0.02%           >=0.9          >=0.9          >=0.9          >=0.9
           0.03%           >=0.9           0.88           0.72           0.64
           0.04%            0.54            0.3           0.36           0.16
           0.042%            0.5           0.16           0.18           0.16
           0.044%           0.32           0.08            0.1            0.1
           0.046%           0.28           0.08           0.04           0.04
           0.048%            0.2           0.04         <=0.02         <=0.02
           0.05%            0.12           0.04         <=0.02         <=0.02
           0.1%           <=0.02         <=0.02         <=0.02         <=0.02

# Препилот для числовой переменной со снижением дисперсии

In [13]:
prepilot_params = PrepilotParams(
    metrics_names=['height_now'],
    injects=[1.0001,1.0002,1.0003,1.0004,1.00042,1.00044,1.00046,1.00048,1.0005,1.001],
    min_group_size=35000, 
    max_group_size=45000, 
    step=5000,
    variance_reduction = ABTest.cuped,
    use_buckets = False,
    stat_test = ABTest.test_hypothesis_boot_confint,
    iterations_number = 50,
    max_beta_score=0.9,
    min_beta_score=0.02,
)

In [14]:
prepilot = PrepilotExperimentBuilder(df, ab_params,
                                     prepilot_params,
                                     split_builder_params)

In [15]:
beta,alpha = prepilot.collect()

INFO:stratification.split_builder:Calculate stratas for guest table
INFO:stratification.binning:27017 outliers found
INFO:stratification.split_builder:control: Desired size = 0.175,             resulting size = 34998, diff = 0.01 %
INFO:stratification.split_builder:target: Desired size = 0.21211864098616987,             resulting size = 35001, diff = 0.0 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_builder:control: Desired size = 0.175,             resulting size = 34998, diff = 0.01 %
INFO:stratification.split_builder:target: Desired size = 0.21211864098616987,             resulting size = 35001, diff = 0.0 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_builder:control: Desired size = 0.175,             resulting size = 34998, diff = 0.01 %
INFO:stratification.split_builder:target: Desired size = 0.21211864098616987,             resulting size = 35001, diff = 0.0 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_bu

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [16]:
beta

split_rate        (35000, 35000) (40000, 40000) (45000, 45000)
metric     MDE                                                
height_now 0.01%             0.8           0.66           0.68
           0.02%            0.36           0.06           0.14
           0.03%          <=0.02         <=0.02         <=0.02
           0.04%          <=0.02         <=0.02         <=0.02
           0.042%         <=0.02         <=0.02         <=0.02
           0.044%         <=0.02         <=0.02         <=0.02
           0.046%         <=0.02         <=0.02         <=0.02
           0.048%         <=0.02         <=0.02         <=0.02
           0.05%          <=0.02         <=0.02         <=0.02
           0.1%           <=0.02         <=0.02         <=0.02

In [17]:
alpha

split_rate,"(35000, 35000)","(40000, 40000)","(45000, 45000)"
metric,,,
height_now,0.02,0.02,0.02


# Препилот для бинарной переменной

In [18]:
df = pd.read_csv('ab_data.csv')

In [20]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    strata_col='country', 
    target='conversion', 
    target_flg='bought', 
    predictors=['weight_now'], 
    numerator='clicks', 
    denominator='sessions', 
    covariate='height_prev', 
    target_prev='height_prev', 
    predictors_prev=['weight_prev'], 
    cluster_col='kl-divergence', 
    clustering_cols=['col1', 'col2', 'col3'], 
    is_grouped=True
)

ab_params = ABTestParams(data_params,simulation_params,hypothesis_params,result_params,splitter_params)

In [21]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': None,
        'target': None
    },
    region_col = "moda_city",
    split_metric_col = "conversion",
    customer_col = "id",
    cols = [],
    cat_cols=[
        
    ],
    pvalue=0.05,
    n_top_cat=100,
    stat_test="ttest_ind"
)

In [22]:
prepilot_params = PrepilotParams(
    metrics_names=['conversion'],
    injects=[1.001,1.002,1.0022,1.0024,1.0026,1.0028,1.003,1.005,1.01],
    min_group_size=35000, 
    max_group_size=55000, 
    step=5000,
    variance_reduction = None,
    use_buckets = False,
    stat_test = ABTest.test_hypothesis_boot_confint,
    iterations_number = 50,
    max_beta_score=0.9,
    min_beta_score=0.01,
)

ab_params = ABTestParams(data_params,simulation_params,hypothesis_params,result_params,splitter_params)

In [23]:
prepilot = PrepilotExperimentBuilder(df, ab_params,
                                     prepilot_params,
                                     split_builder_params)

In [24]:
beta,alpha = prepilot.collect()

INFO:stratification.split_builder:Calculate stratas for guest table
INFO:stratification.binning:36529 outliers found
INFO:stratification.split_builder:control: Desired size = 0.175,             resulting size = 35000, diff = 0.0 %
INFO:stratification.split_builder:target: Desired size = 0.21212121212121213,             resulting size = 35002, diff = 0.01 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_builder:control: Desired size = 0.175,             resulting size = 35000, diff = 0.0 %
INFO:stratification.split_builder:target: Desired size = 0.21212121212121213,             resulting size = 35002, diff = 0.01 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_builder:control: Desired size = 0.175,             resulting size = 35000, diff = 0.0 %
INFO:stratification.split_builder:target: Desired size = 0.21212121212121213,             resulting size = 35002, diff = 0.01 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_bu

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [25]:
beta

split_rate       (35000, 35000) (40000, 40000) (45000, 45000) (50000, 50000)  \
metric     MDE                                                                 
conversion 0.1%           >=0.9          >=0.9          >=0.9          >=0.9   
           0.2%           >=0.9          >=0.9          >=0.9          >=0.9   
           0.22%          >=0.9          >=0.9          >=0.9          >=0.9   
           0.24%          >=0.9          >=0.9          >=0.9          >=0.9   
           0.26%          >=0.9          >=0.9          >=0.9            0.7   
           0.28%          >=0.9            0.6           0.64           0.26   
           0.3%             0.8           0.36           0.32           0.08   
           0.5%          <=0.01         <=0.01         <=0.01         <=0.01   
           1.0%          <=0.01         <=0.01         <=0.01         <=0.01   

split_rate       (55000, 55000)  
metric     MDE                   
conversion 0.1%           >=0.9  
           0.2%           >=0.9  
           0.22%          >=0.9  
           0.24%            0.6  
           0.26%           0.38  
           0.28%           0.14  
           0.3%            0.02  
           0.5%          <=0.01  
           1.0%          <=0.01

In [26]:
alpha

split_rate,"(35000, 35000)","(40000, 40000)","(45000, 45000)","(50000, 50000)","(55000, 55000)"
metric,,,,,
conversion,0.0,0.0,0.0,0.0,0.0
